In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input -? OUtput? RF
        self.bn1 = nn.BatchNorm2d(8)
        self.do1 = nn.Dropout2d(0.1)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.do2 = nn.Dropout2d(0.1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.transition1 = nn.Conv2d(16, 8, kernel_size=1)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(8)
        self.do3 = nn.Dropout2d(0.1)
        self.conv4 = nn.Conv2d(8, 16, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(16)
        self.do4 = nn.Dropout2d(0.1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.transition2 = nn.Conv2d(16, 8, kernel_size=1)
        self.conv5 = nn.Conv2d(8, 8, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(8)
        self.do5 = nn.Dropout2d(0.1)
        self.conv6 = nn.Conv2d(8, 8, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(8)
        self.fc1 = nn.Linear(8 * 7 * 7, 32)  # Assuming MNIST input size of 28x28
        self.dropout_fc = nn.Dropout(0.1)
        self.fc2 = nn.Linear(32, 32)
        self.dropout_fc = nn.Dropout(0.1)
        self.fc3 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool1(self.do2(self.bn2(F.relu(self.conv2(self.do1(self.bn1(F.relu(self.conv1(x)))))))))
        x = self.transition1(x)
        x = self.pool2(self.do4(self.bn4(F.relu(self.conv4(self.do3(self.bn3(F.relu(self.conv3(x)))))))))
        x = self.transition2(x)
        x = self.bn6(F.relu(self.conv6(self.do5(self.bn5(F.relu(self.conv5(x)))))))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout_fc(x)
        x = F.relu(self.fc2(x))
        x = self.dropout_fc(x)
        x = self.fc3(x)
        return x

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         Dropout2d-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,168
       BatchNorm2d-5           [-1, 16, 28, 28]              32
         Dropout2d-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8            [-1, 8, 14, 14]             136
            Conv2d-9            [-1, 8, 14, 14]             584
      BatchNorm2d-10            [-1, 8, 14, 14]              16
        Dropout2d-11            [-1, 8, 14, 14]               0
           Conv2d-12           [-1, 16, 14, 14]           1,168
      BatchNorm2d-13           [-1, 16, 14, 14]              32
        Dropout2d-14           [-1, 16,

In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

for epoch in range(1, 20):
    print("## Epoch " + str(epoch) + ":")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    #scheduler.step()

## Epoch 1:


loss=2.205014944076538 batch_id=22:   5%|▍         | 23/469 [00:03<01:10,  6.36it/s]


KeyboardInterrupt: 